# IMPORT FUNCFIONS

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import pickle
import os
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import average_precision_score
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder, WeightedL1Embedder, WeightedL2Embedder
from sklearn.linear_model import LogisticRegression

# READ DATA

In [2]:
def get_edges_dict(traindata):
    if os.path.isfile("edgesdic.pkl"):
        f = open("edgesdic.pkl","rb")
        edges = pickle.load(f)
        f.close()
        return edges
    else:
        edges = dict()
        for edge in range(len(traindata)): 
            edges[(traindata.loc[[edge]]['id_1'].values[0],traindata.loc[[edge]]['id_2'].values[0])] = 1 
            edges[(traindata.loc[[edge]]['id_2'].values[0],traindata.loc[[edge]]['id_1'].values[0])] = 1  
        f = open("edgesdic.pkl","wb")
        pickle.dump(edges,f)
        f.close()
        return edges

def get_negative_edges(traindata,g):
    edges = get_edges_dict(traindata)
    # for missing edges.
    negative_edges = set([])
    maxNodenum = max(max(traindata['id_1']),max(traindata['id_2']))
    #產生與原圖同edge數目的negative edges
    while (len(negative_edges)<len(traindata)):
        node1=random.randint(0, maxNodenum) 
        node2=random.randint(0, maxNodenum) 
        tmp = edges.get((node1,node2),0) #edge不在graph裡標為0
        if tmp == 0 and node1!=node2 and g.has_node(node1) and g.has_node(node2): # if edge不在graph裡才要做處理
            negative_edges.add((node1,node2))
        else:
            continue
    return negative_edges

# MODELS

In [3]:
def get_proximity_score(G, edges, feature):
    jc_list = [[] for i in itertools.repeat(None, len(edges))]
    pa_list = [[] for i in itertools.repeat(None, len(edges))]
    adar_list = [[] for i in itertools.repeat(None, len(edges))]
    
    jc = nx.jaccard_coefficient(G, edges)
    pa = nx.preferential_attachment(G, edges)
    aa = nx.adamic_adar_index(G, edges)

    for i, data in enumerate(jc):
        jc_list[i] = data[2]
    for i, data in enumerate(pa):
        pa_list[i] = data[2]
    for i, data in enumerate(aa):
        adar_list[i] = data[2]  
    
    if feature == 'jc':
        return jc_list
    elif feature == 'pa':
        return pa_list
    elif feature == 'aa':
        return adar_list
    else:
        return jc_list, pa_list, adar_list
    
    return jc_list, pa_list, adar_list

In [4]:
all_ap = []
all_auc = []

filename = 'citeseer'

for i in range(10):
    ori_df = pd.read_csv('data/'+filename+'.txt', header=None, sep=' ')
    ori_df.columns = ['id_1', 'id_2']
    
    nodes_number = 3327
    ori_graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(ori_df.shape[0]):
        ori_graph_np[ori_df.iloc[i, 0], ori_df.iloc[i, 1]] = 1
        ori_graph_np[ori_df.iloc[i, 1], ori_df.iloc[i, 0]] = 1

    ori_G = nx.from_numpy_matrix(ori_graph_np)
    
    
    #ori_G = nx.from_pandas_edgelist(ori_df, 'id_1', 'id_2')
    #print(nx.info(ori_G))
    
    negative_edges = get_negative_edges(ori_df, ori_G)
    #print(len(negative_edges))

    train_df, test_df = train_test_split(ori_df, test_size=0.1)
    
    nodes_number = len(ori_G.nodes)
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(train_df.shape[0]):
        graph_np[train_df.iloc[i, 0], train_df.iloc[i, 1]] = 1
        graph_np[train_df.iloc[i, 1], train_df.iloc[i, 0]] = 1

    train_G = nx.from_numpy_matrix(graph_np)
    #print(nx.info(train_G))

    pos_edges = [(data.id_1, data.id_2) for data in train_df.itertuples()]

    train_df['jc'], train_df['pa'], train_df['aa'] = get_proximity_score(train_G, pos_edges, 'all')

    test_pos_edges = [(data.id_1, data.id_2) for data in test_df.itertuples()]
    test_df['jc'], test_df['pa'], test_df['aa'] = get_proximity_score(train_G, test_pos_edges,'all')

    df_neg = pd.DataFrame(list(negative_edges), columns=['id_1', 'id_2'])
    neg_edges = list(negative_edges)

    df_neg['jc'], df_neg['pa'], df_neg['aa'] = get_proximity_score(train_G, neg_edges, 'all')

    X_train_neg, X_test_neg = train_test_split(df_neg, test_size=0.1)

    y_train_pos = np.ones(len(train_df))
    y_train_neg = np.zeros(len(X_train_neg))
    y_test_pos = np.ones(len(test_df))
    y_test_neg = np.zeros(len(X_test_neg))

    X_train = np.concatenate((train_df, X_train_neg))
    y_train = np.concatenate((y_train_pos, y_train_neg))
    X_test = np.concatenate((test_df, X_test_neg))
    y_test = np.concatenate((y_test_pos, y_test_neg))

    clf1 = RandomForestClassifier(n_estimators=400)
    clf1.fit(X_train, y_train)

    predict_Y = clf1.predict(X_test)
    
    ap = average_precision_score(y_test, predict_Y)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(y_test, predict_Y, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    all_ap.append(ap)
    all_auc.append(auc)

print("####################### SUM UP ############################")
print("AP MEAN : ", np.array(all_ap).mean())
print("AP STD : ", np.array(all_ap).std())
print("AUC MEAN : ", np.array(all_auc).mean())
print("AUC STD : ", np.array(all_auc).std())

AP：  0.6546370780564794
AUC SCORE:  0.7096597145993414
AP：  0.6700447673492853
AUC SCORE:  0.7233809001097695
AP：  0.6693539045391612
AUC SCORE:  0.7239297475301866
AP：  0.6792760299895317
AUC SCORE:  0.7321624588364436
AP：  0.6509651291362439
AUC SCORE:  0.7069154774972558
AP：  0.6629412628384851
AUC SCORE:  0.716794731064764
AP：  0.6590979406566673
AUC SCORE:  0.712403951701427
AP：  0.66761544941669
AUC SCORE:  0.7206366630076839
AP：  0.6770049643616858
AUC SCORE:  0.7332601536772777
AP：  0.6861880866506866
AUC SCORE:  0.7392974753018662
####################### SUM UP ############################
AP MEAN :  0.6677124612994916
AP STD :  0.010573419253250118
AUC MEAN :  0.7218441273326015
AUC STD :  0.0101760515344253
